# Notebook Objective

The objective of the notebook is to study and implement various tokenizers that are prominent in the literature. Implemented two tokenizers:

- Simple Regex Based Tokenizer __(WORD LEVEL)__
  
    This tokenizer splits the input text into tokens based on simple regular expressions, typically at word boundaries, such as spaces, punctuation marks, and special characters. It requires domain knowledge to effectively convert raw text into vocabulary (which is rule based / heurestic driven)

- BPE (Byte Pair encoding) Tokenizer __(SUBWORD LEVEL)__

    This tokenizer operates at the subword level and uses an algorithm to iteratively merge the most frequent pairs of bytes (or characters) in the text. This technique helps to break down words into smaller, more manageable subword units, improving the tokenizer's ability to handle out-of-vocabulary words and languages with rich morphology


There is also an additional tokenizer that is used which is character tokenizer which breaks the input text into individual characters, treating each character as a token. Character tokenization is useful for languages with complex morphology or in cases where fine-grained tokenization is needed, such as in character-level language models or spelling correction tasks but it also adds an additional overhead since more number of tokens is required to represent same sentence but the deep learning models built subsequently can required huge resource if the length of sentence is too long. This notebook doesn't have character tokenizer.

## Importing Libraries

In [1]:
import os
import pathlib
import regex as re
%reload_ext autoreload
%autoreload 2

## Configuration

In [2]:
class Config:
    def __init__(self):
        self.dataset_path = pathlib.Path('./datasets')
        self.dataset_file = 'the-verdict.txt'

cfg = Config()

## Loading Data

In [3]:
with open(os.path.join(cfg.dataset_path, cfg.dataset_file), 'r') as file:
    # Read but ignore lines with only new line character
    data = [line for line in file.readlines() if line != '\n']

raw_data = ' '.join(data)
print(f'Total number of characters in the data: {len(raw_data)}')
print(f'Total number of unique characters in the data: {len(set(raw_data))}')
print(raw_data[:400])

Total number of characters in the data: 20479
Total number of unique characters in the data: 62
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)
 "The height of his glory"--that was what the women called it. I can hear


## Regex based tokenizer

Regex based tokenizer involves finding a regular expression that can effectively convert your raw data into tokens. The common idea is to split your raw data into words where words are found out by splitting based on some heurestics like splitting based on spaces, tabs, new line, exclamation point. By extracting unique words from your split raw text, a vocabulary of words/tokens is build that is then assigned a unique integer that is used to map words/tokens to token idx.

It is a design based tokenizer where domain knowledge and the required task that is required to split the raw data. For eg. If working with code generator tabs and spaces can't be neglected and will require special tokens in vocabulary for them. 

__NOTE:__ Also special tokens (like <unk> or |unk|) might be required in case some word that is not present in vocabulary is encountered

### Building Vocabulary

In [4]:
def text_to_tokens(raw_text, split_expression):
    return re.split(split_expression, raw_text)

In [5]:
split_expression = r'(\s+)' # split based on spaces
tokens = text_to_tokens(raw_data, split_expression)
print(tokens[:200])
print(f'Number of tokens: {len(tokens)}')

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius--though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough--so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that,', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory,', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting,', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow,', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on', ' ', 'the', ' ', 'Riviera.', ' ', '(Though', ' ', 'I', ' ', 'rather', ' ', 'thought', ' ', 'it', ' ', 'would', ' ', 'have', ' ', 'been', ' ', 'Rome', ' ', 'or', ' ', 'Florence.)', '\n ', '"The', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory"--that', ' ', 'was', ' ', 'what', ' ', 'the', ' ', 'women', ' ', 'called', ' ', 'it.', ' ', 'I', ' ', 'can', ' ', 'hear', ' ', 'Mrs.', ' ', 

In [6]:
split_expression = r'(\s+|\n)' # split based on spaces and new line
tokens = text_to_tokens(raw_data, split_expression)
print(tokens[:200])
print(f'Number of tokens: {len(tokens)}')

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius--though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough--so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that,', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory,', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting,', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow,', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on', ' ', 'the', ' ', 'Riviera.', ' ', '(Though', ' ', 'I', ' ', 'rather', ' ', 'thought', ' ', 'it', ' ', 'would', ' ', 'have', ' ', 'been', ' ', 'Rome', ' ', 'or', ' ', 'Florence.)', '\n ', '"The', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory"--that', ' ', 'was', ' ', 'what', ' ', 'the', ' ', 'women', ' ', 'called', ' ', 'it.', ' ', 'I', ' ', 'can', ' ', 'hear', ' ', 'Mrs.', ' ', 

In [7]:
split_expression = r'(-|--|---|\s+|\n)' # split based on spaces, new line and dashes (em dash, en dash, hyphen)
tokens = text_to_tokens(raw_data, split_expression)
print(tokens[:200])
print(f'Number of tokens: {len(tokens)}')

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '-', '', '-', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '-', '', '-', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that,', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory,', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting,', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow,', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on', ' ', 'the', ' ', 'Riviera.', ' ', '(Though', ' ', 'I', ' ', 'rather', ' ', 'thought', ' ', 'it', ' ', 'would', ' ', 'have', ' ', 'been', ' ', 'Rome', ' ', 'or', ' ', 'Florence.)', '\n ', '"The', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory"', '-', '', '-', 'that', ' ', 'was', ' ', 'what', ' ', 'the', ' ', 'women', ' ', 'called', ' ', 'it.', ' ', 

In [8]:
split_expression = r'([,.:;?!_\(\)\"\']|-|--|---|\s+|\n)' # Including punctuations as well
tokens = text_to_tokens(raw_data, split_expression)
print(tokens[:200])
print(f'Number of tokens: {len(tokens)}')

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '-', '', '-', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '-', '', '-', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', '', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', ',', '', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting', ',', '', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow', ',', '', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on', ' ', 'the', ' ', 'Riviera', '.', '', ' ', '', '(', 'Though', ' ', 'I', ' ', 'rather', ' ', 'thought', ' ', 'it', ' ', 'would', ' ', 'have', ' ', 'been', ' ', 'Rome', ' ', 'or', ' ', 'Florence', '.', '', ')', '', '\n ', '', '"', 'The', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', '"', '', '-', '', '-', 'that', ' '

In [9]:
# Dropping tokens with only spaces, other pronounciation add some context as well
tokens_cleaned = [token.strip() for token in tokens if token.strip()]
print(tokens_cleaned[:50])
print(f'Number of tokens: {len(tokens_cleaned)}')

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '-', '-', 'though', 'a', 'good', 'fellow', 'enough', '-', '-', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and']
Number of tokens: 4863


In [10]:
# combining dashes if consecutive for em and en dash
final_tokens = []
i = 0
while i < (len(tokens_cleaned) - 1):
    if tokens_cleaned[i] == '-' and tokens_cleaned[i+1] == '-':
        if i+2 < len(tokens_cleaned) and tokens_cleaned[i+2] == '-':
            final_tokens.append('---')
            i = i+2
        else:
            final_tokens.append('--')
            i = i+1
    else:
        final_tokens.append(tokens_cleaned[i])
    i = i+1
print(final_tokens[:50])
print(f'Number of tokens: {len(final_tokens)}')

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']
Number of tokens: 4765


### Encoding/Decoding text to token index

In [11]:
vocab = sorted(set(final_tokens))
print(f'Number of unique tokens: {len(vocab)}')
print(vocab[-5:])
tokens_to_idx = {token:idx for idx, token in enumerate(vocab)}
tokens_to_idx['|unk|'] = len(tokens_to_idx)
idx_to_token = {idx:token for token, idx in tokens_to_idx.items()}

Number of unique tokens: 1140
['yet', 'you', 'younger', 'your', 'yourself']


In [12]:
def post_split_processing(tokens):
    tokens_cleaned = [token.strip() for token in tokens if token.strip()]
    final_tokens = []
    i = 0
    while i < (len(tokens_cleaned) - 1):
        if tokens_cleaned[i] == '-' and tokens_cleaned[i+1] == '-':
            if i+2 < len(tokens_cleaned) and tokens_cleaned[i+2] == '-':
                final_tokens.append('---')
                i = i+2
            else:
                final_tokens.append('--')
                i = i+1
        else:
            final_tokens.append(tokens_cleaned[i])
        i = i+1
    return final_tokens

In [13]:
sentence = 'I HAD always thought Jack Gisburn' # Test Sentence
final_split_expression= r'([,.:;?_\(\)\"\']|-|--|---|\s+|\n)'
sentence_tokens = post_split_processing(text_to_tokens(sentence, final_split_expression))
print(sentence_tokens)
sentence_tokens = [tokens_to_idx[word_token] for word_token in sentence_tokens] # Doesn't handle if token is not present
print(sentence_tokens)
decoded_sentence = ' '.join([idx_to_token[token_idx] for token_idx in sentence_tokens])
print(decoded_sentence)

['I', 'HAD', 'always', 'thought', 'Jack']
[54, 45, 150, 1013, 58]
I HAD always thought Jack


In [14]:
sentence = 'My name is Andrew.' # Test Sentence
final_split_expression= r'([,.:;?_\(\)\"\']|-|--|---|\s+|\n)'
sentence_tokens = post_split_processing(text_to_tokens(sentence, final_split_expression))
print(sentence_tokens)
sentence_tokens = [tokens_to_idx.get(word_token, tokens_to_idx['|unk|']) for word_token in sentence_tokens]
print(sentence_tokens)
decoded_sentence = ' '.join([idx_to_token[token_idx] for token_idx in sentence_tokens])
print(decoded_sentence)

['My', 'name', 'is', 'Andrew']
[69, 1140, 588, 1140]
My |unk| is |unk|


### Trying our tokenizer class functionality

In [15]:
from tokenizer.simple_tokenizer import RegexTokenizer
split_regex= r'([,.:;?!_\(\)\"\']|-|--|---|\s+|\n)'
tokenizer = RegexTokenizer(raw_data, split_regex, special_tokens={'|unk|', }, unknown_token='|unk|')

########## Build Vocabulary ##########


In [16]:
sentence = 'My name is Andrew.'
encoded_sentence = tokenizer.encode(sentence)
print(encoded_sentence)
decoded_sentence = tokenizer.decode(encoded_sentence)
print(decoded_sentence)

[69, 1140, 588, 1140]
My |unk| is |unk|


## Byte Pair Encoding

This tokenizer is subword tokenizer which involves encoding a text in its byte representation (typically done using `utf-8`). Following which, algorithm iteratively combines tokens of maximum frequency to form new tokens till a maximum vocabulary size is reached. It allows representation of many languages (via `utf-8`) and doesn't involve any heurestic based design mechanism for tokenization but depends entirely on the corpus of data available for combining tokens.

Doesn't need special <span style="color:red">unknown</span> token since it's a subword tokenizer and every character is present in vocabulary so it can break a word, not seen in training data, into corresponding character and tokenize it.

### Building Vocabulary

In [17]:
sample_sentence = raw_data[:200]
print(f"Length: ({len(sample_sentence)}) |||| {sample_sentence}")
byte_representation = sample_sentence.encode('utf-8')
print(byte_representation)
tokens = list(map(int, byte_representation))
print(f"Length: ({len(sample_sentence)}) |||| {tokens}")

Length: (200) |||| I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a
b'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a'
Length: (200) |||| [73, 32, 72, 65, 68, 32, 97, 108, 119, 97, 121, 115, 32, 116, 104, 111, 117, 103, 104, 116, 32, 74, 97, 99, 107, 32, 71, 105, 115, 98, 117, 114, 110, 32, 114, 97, 116, 104, 101, 114, 32, 97, 32, 99, 104, 101, 97, 112, 32, 103, 101, 110, 105, 117, 115, 45, 45, 116, 104, 111, 117, 103, 104, 32, 97, 32, 103, 111, 111, 100, 32, 102, 101, 108, 108, 111, 119, 32, 101, 110, 111, 117, 103, 104, 45, 45, 115, 111, 32, 105, 116, 32, 119, 97, 115, 32, 110, 111, 32, 103, 114, 101, 97, 116, 32, 115, 117, 114, 112, 114, 105, 115, 101, 32, 116, 111, 32, 109, 101, 32,

In [18]:
def find_max_frequency_pair(token_ids):
    char_counts = {}
    for char1, char2 in zip(token_ids, token_ids[1:]):
        char_counts[(char1, char2)] = char_counts.get((char1, char2), 0) + 1
    
    char_counts = max(char_counts.items(), key=lambda x: x[1])
    return char_counts

def combine_characters(token_ids, pair_combine, token_id):
    idx = 0
    new_tokens_list = []
    while idx < (len(token_ids) - 1):
        if (token_ids[idx] == pair_combine[0]) and (token_ids[idx+1] == pair_combine[1]):
            new_tokens_list.append(token_id)
            idx+=1
        else:
            new_tokens_list.append(token_ids[idx])
        idx+=1
    # If missed last token
    if idx == len(token_ids) - 1:
        new_tokens_list.append(token_ids[idx])
    return new_tokens_list

print(f"{'='*20} TOKENS {'='*20}")
print(tokens)
print(f"Number of tokens before BPE: {len(tokens)}")

new_tokens = tokens[:]
new_token_idx = 256
token_to_idx = {}
vocabulary_size = 300

for i in range(vocabulary_size - 256):
    max_freq_tokens = find_max_frequency_pair(new_tokens)[0]
    # print(f'Merging {max_freq_tokens}')
    new_tokens = combine_characters(new_tokens, max_freq_tokens, new_token_idx+i)
    token_to_idx[max_freq_tokens] = new_token_idx+i
print(f"{'='*20} NEW TOKENS {'='*20}")
print(new_tokens)
print(f"Number of tokens after BPE: {len(new_tokens)}")
print(f"Compression Ratio: {len(tokens)/len(new_tokens):.2f}x")

==================== TOKENS ====================
[73, 32, 72, 65, 68, 32, 97, 108, 119, 97, 121, 115, 32, 116, 104, 111, 117, 103, 104, 116, 32, 74, 97, 99, 107, 32, 71, 105, 115, 98, 117, 114, 110, 32, 114, 97, 116, 104, 101, 114, 32, 97, 32, 99, 104, 101, 97, 112, 32, 103, 101, 110, 105, 117, 115, 45, 45, 116, 104, 111, 117, 103, 104, 32, 97, 32, 103, 111, 111, 100, 32, 102, 101, 108, 108, 111, 119, 32, 101, 110, 111, 117, 103, 104, 45, 45, 115, 111, 32, 105, 116, 32, 119, 97, 115, 32, 110, 111, 32, 103, 114, 101, 97, 116, 32, 115, 117, 114, 112, 114, 105, 115, 101, 32, 116, 111, 32, 109, 101, 32, 116, 111, 32, 104, 101, 97, 114, 32, 116, 104, 97, 116, 44, 32, 105, 110, 32, 116, 104, 101, 32, 104, 101, 105, 103, 104, 116, 32, 111, 102, 32, 104, 105, 115, 32, 103, 108, 111, 114, 121, 44, 32, 104, 101, 32, 104, 97, 100, 32, 100, 114, 111, 112, 112, 101, 100, 32, 104, 105, 115, 32, 112, 97, 105, 110, 116, 105, 110, 103, 44, 32, 109, 97, 114, 114, 105, 101, 100, 32, 97]
Number of tokens 

### Encoding/Decoding text to token index

In [19]:
idx_to_vocab = {idx:bytes([idx]) for idx in range(256)}
for pair,idx in token_to_idx.items():
    idx_to_vocab[idx] = idx_to_vocab[pair[0]] + idx_to_vocab[pair[1]]

In [20]:
def decode(token_ids):
    tokens = b"".join(idx_to_vocab[idx] for idx in token_ids)
    return tokens.decode('utf-8', errors='ignore')
decode(new_tokens)

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a'

In [32]:
def encode(token_ids):
    new_tokens = list(token_ids)
    for pair,token_idx in token_to_idx.items():
        new_tokens = combine_characters(new_tokens, pair, token_idx)
    return new_tokens
encoded_tokens = encode(tokens)
print(encoded_tokens)
print(f"Number of encoded tokens: {len(encoded_tokens)}")
print(decode(encoded_tokens))

[299, 98, 271, 110, 32, 114, 272, 256, 114, 258, 32, 99, 273, 112, 262, 274, 105, 117, 115, 275, 116, 104, 264, 258, 262, 111, 111, 265, 102, 101, 108, 276, 119, 32, 274, 264, 275, 115, 266, 105, 260, 269, 115, 32, 110, 111, 262, 114, 101, 97, 260, 115, 271, 112, 114, 261, 278, 109, 278, 273, 114, 270, 272, 267, 268, 257, 279, 256, 105, 259, 260, 111, 102, 32, 280, 262, 276, 114, 121, 267, 279, 104, 97, 265, 100, 114, 111, 112, 112, 101, 265, 280, 32, 112, 97, 268, 116, 268, 103, 267, 109, 97, 114, 114, 105, 101, 100, 258]
Number of encoded tokens: 108
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a


### Trying our tokenizer class functionality

In [52]:
from tokenizer.byte_pair_tokenizer import BytePairTokenizer
sample_sentence = raw_data[:200]
vocabulary_size = 300 - 256
bpe = BytePairTokenizer(sample_sentence, vocabulary_size)

In [63]:
print(bpe.encode(sample_sentence))
print(bpe.encode(sample_sentence) == encoded_tokens)
print(bpe.decode(bpe.encode(sample_sentence)))

[299, 98, 271, 110, 32, 114, 272, 256, 114, 258, 32, 99, 273, 112, 262, 274, 105, 117, 115, 275, 116, 104, 264, 258, 262, 111, 111, 265, 102, 101, 108, 276, 119, 32, 274, 264, 275, 115, 266, 105, 260, 269, 115, 32, 110, 111, 262, 114, 101, 97, 260, 115, 271, 112, 114, 261, 278, 109, 278, 273, 114, 270, 272, 267, 268, 257, 279, 256, 105, 259, 260, 111, 102, 32, 280, 262, 276, 114, 121, 267, 279, 104, 97, 265, 100, 114, 111, 112, 112, 101, 265, 280, 32, 112, 97, 268, 116, 268, 103, 267, 109, 97, 114, 114, 105, 101, 100, 258]
True
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a


In [68]:
print(bpe.encode('नमस्ते')) # Can encode things it has not seen during training
encoding_hindi = bpe.encode('नमस्ते')
print(f'Decoded word is: {bpe.decode(encoding_hindi)}')

[224, 164, 168, 224, 164, 174, 224, 164, 184, 224, 165, 141, 224, 164, 164, 224, 165, 135]
Decoded word is: नमस्ते
